

# ConfigMaps and Secrets

##  Configuring containerized applications

## Passing command-line arguments to containers

In all the examples so far, you’ve created containers that ran the default command defined in the container image, but Kubernetes allows overriding the command as part of the pod’s container definition when you want to run a different executable instead of the one specified in the image, or want to run it with a different set of command-line arguments. We’ll look at how to do that now.

###  Defining the command and arguments in Docker

The first thing I need to explain is that the whole command that gets executed in the container is composed of two parts: the command and the arguments.

**Understanding ENTRYPOINT and CMD**

In a Dockerfile, two instructions define the two parts:
- ENTRYPOINT defines the executable invoked when the container is started.
- CMD specifies the arguments that get passed to the ENTRYPOINT.

Although you can use the CMD instruction to specify the command you want to execute when the image is run, the correct way is to do it through the ENTRYPOINT instruction and to only specify the CMD if you want to define the default arguments. The image can then be run without specifying any arguments

    $ docker run <image>

or with additional arguments, which override whatever’s set under CMD in the Dockerfile:

    $ docker run <image> <arguments>

**Understanding the difference between the shell and exec forms**

But there’s more. Both instructions support two different forms:
- `shell` form—For example, `ENTRYPOINT node app.js`.
- `exec` form—For example, `ENTRYPOINT ["node", "app.js"]`.

The difference is whether the specified command is invoked inside a shell or not.

In the kubia image you created in chapter 2, you used the exec form of the ENTRYPOINT instruction:

    ENTRYPOINT ["node", "app.js"]

This runs the node process directly (not inside a shell), as you can see by listing the processes running inside the container:

    $ docker exec 4675d ps x
      PID TTY      STAT   TIME COMMAND
        1 ?        Ssl    0:00 node app.js
       12 ?        Rs     0:00 ps x



If you’d used the shell form (ENTRYPOINT node app.js), these would have been the container’s processes:

    $ docker exec -it e4bad ps x
      PID TTY      STAT   TIME COMMAND
        1 ?        Ss     0:00 /bin/sh -c node app.js
        7 ?        Sl     0:00 node app.js
       13 ?        Rs+    0:00 ps x



As you can see, in that case, the main process (PID 1) would be the shell process instead of the node process. The node process (PID 7) would be started from that shell. The shell process is unnecessary, which is why you should always use the exec form of the ENTRYPOINT instruction.

**Making the interval configurable in your fortune image**

Let’s modify your fortune script and image so the delay interval in the loop is configurable. You’ll add an INTERVAL variable and initialize it with the value of the first command-line argument, as shown in the following listing.

    #!/bin/bash
    trap "exit" SIGINT
    INTERVAL=$1
    echo Configured to generate new fortune every $INTERVAL seconds
    mkdir -p /var/htdocs
    while :
    do
      echo $(date) Writing fortune to /var/htdocs/index.html
      /usr/games/fortune > /var/htdocs/index.html
      sleep $INTERVAL
    done



You’ve added or modified the lines in bold font. Now, you’ll modify the Dockerfile so it uses the exec version of the ENTRYPOINT instruction and sets the default interval to 10 seconds using the CMD instruction, as shown in the following listing.

    FROM ubuntu:latest
    RUN apt-get update ; apt-get -y install fortune
    ADD fortuneloop.sh /bin/fortuneloop.sh
    RUN chmod +x /bin/fortuneloop.sh
    ENTRYPOINT ["/bin/fortuneloop.sh"]
    CMD ["10"]

You can now build and push the image to Docker Hub. This time, you’ll tag the image as args instead of latest:

    docker build -t docker.io/leon11sj/fortune:args .
    docker push docker.io/leon11sj/fortune:args

You can test the image by running it locally with Docker:

    docker run -it docker.io/leon11sj/fortune:args

And you can override the default sleep interval by passing it as an argument:

    docker run -it docker.io/leon11sj/fortune:args 3

Now that you’re sure your image honors the argument passed to it, let’s see how to use it in a pod.

### Overriding the command and arguments in Kubernetes

In Kubernetes, when specifying a container, you can choose to override both `ENTRYPOINT` and `CMD`. To do that, you set the properties `command` and `args` in the container specification, as shown in the following listing.

    kind: Pod
    spec:
      containers:
      - image: some/image
        command: ["/bin/command"]
        args: ["arg1", "arg2", "arg3"]



In most cases, you’ll only set custom arguments and rarely override the command (except in general-purpose images such as busybox, which doesn’t define an ENTRYPOINT at all).

> NOTE: The command and args fields can’t be updated after the pod is created.

The two Dockerfile instructions and the equivalent pod spec fields are shown in table 7.1.

<table cellpadding="8" cellspacing="5" frame="hsides" rules="groups" width="100%"> 
  <colgroup span="3"> 
   <col width="200"> 
   <col width="200"> 
   <col width="200"> 
  </colgroup> 
  <thead> 
   <tr> 
    <th scope="col" valign="top"> <p>Docker</p> </th> 
    <th scope="col" valign="top"> <p>Kubernetes</p> </th> 
    <th scope="col" valign="top"> <p>Description</p> </th> 
   </tr> 
  </thead> 
  <tbody> 
   <tr> 
    <td>ENTRYPOINT</td> 
    <td>command</td> 
    <td>The executable that’s executed inside the container</td> 
   </tr> 
   <tr> 
    <td>CMD</td> 
    <td>args</td> 
    <td>The arguments passed to the executable</td> 
   </tr> 
  </tbody> 
 </table>

**Running the fortune pod with a custom interval**

To run the fortune pod with a custom delay interval, you’ll copy your fortune-pod.yaml into fortune-pod-args.yaml and modify it as shown in the following listing.

```yml
apiVersion: v1
kind: Pod
metadata:
  name: fortune2s
spec:
  containers:
  - image: leon11sj/fortune:args
    args: ["2"]
    name: html-generator
    volumeMounts:
    - name: html
      mountPath: /var/htdocs
  - image: nginx:alpine
    name: web-server
    volumeMounts:
    - name: html
      mountPath: /usr/share/nginx/html
      readOnly: true
    ports:
    - containerPort: 80
      protocol: TCP
  volumes:
  - name: html
    emptyDir: {}
```

    kubectl apply -f ex01-fortune-pod-args.yaml

You added the args array to the container definition. Try creating this pod now. The values of the array will be passed to the container as command-line arguments when it is run.

The array notation used in this listing is great if you have one argument or a few. If you have several, you can also use the following notation:

    args:
        - foo
        - bar
        - "15"

> TIP: You don’t need to enclose string values in quotations marks (but you must enclose numbers).

Specifying arguments is one way of passing config options to your containers through command-line arguments. Next, you’ll see how to do it through environment variables.

    kubectl logs fortune2s -c html-generator

    kubectl delete pod fortune2s

## Setting environment variables for a container

**Making the interval in your fortune image configurable through an environment variable**

Let’s see how to modify your fortuneloop.sh script once again to allow it to be configured from an environment variable, as shown in the following listing.

    #!/bin/bash
    trap "exit" SIGINT
    echo Configured to generate new fortune every $INTERVAL seconds
    mkdir -p /var/htdocs
    while :
    do
      echo $(date) Writing fortune to /var/htdocs/index.html
      /usr/games/fortune > /var/htdocs/index.html
      sleep $INTERVAL
    done

All you had to do was remove the row where the INTERVAL variable is initialized. Because your “app” is a simple bash script, you didn’t need to do anything else. If the app was written in Java you’d use System.getenv("INTERVAL"), whereas in Node.JS you’d use process.env.INTERVAL, and in Python you’d use os.environ['INTERVAL'].

    docker build -t leon11sj/fortune:env .
    docker push leon11sj/fortune:env

### Specifying environment variables in a container definition

After building the new image (I’ve tagged it as luksa/fortune:env this time) and pushing it to Docker Hub, you can run it by creating a new pod, in which you pass the environment variable to the script by including it in your container definition, as shown in the following listing.

```yml
apiVersion: v1
kind: Pod
metadata:
  name: fortune-env
spec:
  containers:
  - image: leon11sj/fortune:env
    env:
    - name: INTERVAL
      value: "30"
    name: html-generator
    volumeMounts:
    - name: html
      mountPath: /var/htdocs
  - image: nginx:alpine
    name: web-server
    volumeMounts:
    - name: html
      mountPath: /usr/share/nginx/html
      readOnly: true
    ports:
    - containerPort: 80
      protocol: TCP
  volumes:
  - name: html
    emptyDir: {}
```

As mentioned previously, you set the environment variable inside the container definition, not at the pod level.

> NOTE: Don’t forget that in each container, Kubernetes also automatically exposes environment variables for each service in the same namespace. These environment variables are basically auto-injected configuration.

    kubectl apply -f ex02-fortune-pod-env.yaml

    kubectl logs fortune-env -c html-generator

Having values effectively hardcoded in the pod definition means you need to have separate pod definitions for your production and your development pods. To reuse the same pod definition in multiple environments, it makes sense to decouple the configuration from the pod descriptor. Luckily, you can do that using a ConfigMap resource and using it as a source for environment variable values using the valueFrom instead of the value field.

## Decoupling configuration with a ConfigMap

The whole point of an app’s configuration is to keep the config options that vary between environments, or change frequently, separate from the application’s source code. If you think of a pod descriptor as source code for your app (and in microservices architectures that’s what it really is, because it defines how to compose the individual components into a functioning system), it’s clear you should move the configuration out of the pod description.

### Introducing ConfigMaps

Kubernetes allows separating configuration options into a separate object called a ConfigMap, which is a map containing key/value pairs with the values ranging from short literals to full config files.

An application doesn’t need to read the ConfigMap directly or even know that it exists. The contents of the map are instead passed to containers as either environment variables or as files in a volume (see figure 7.2). And because environment variables can be referenced in command-line arguments using the `$(ENV_VAR)` syntax, you can also pass ConfigMap entries to processes as command-line arguments.

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/07fig02.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_7-2.png" class="medium-zoom-image">

Sure, the application can also read the contents of a ConfigMap directly through the Kubernetes REST API endpoint if needed, but unless you have a real need for this, you should keep your app Kubernetes-agnostic as much as possible.

Regardless of how an app consumes a ConfigMap, having the config in a separate standalone object like this allows you to keep multiple manifests for ConfigMaps with the same name, each for a different environment (development, testing, QA, production, and so on). Because pods reference the ConfigMap by name, you can use a different config in each environment while using the same pod specification across all of them (see figure 7.3).

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/07fig03_alt.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_7-3.png" class="medium-zoom-image">

### Creating a ConfigMap

**Using the kubectl create configmap command**

Nothing extraordinary. You could easily have written this YAML yourself (you wouldn’t need to specify anything but the name in the metadata section, of course) and posted it to the Kubernetes API with the well-known

```yml
apiVersion: v1
kind: ConfigMap
metadata:
  name: fortune-config
data:
  sleep-interval: "25"
```

    $ kubectl apply -f fortune-config.yaml



**Creating a ConfigMap entry from the contents of a file**

ConfigMaps can also store coarse-grained config data, such as complete config files. To do this, the kubectl create configmap command also supports reading files from disk and storing them as individual entries in the ConfigMap:

    $ kubectl create configmap my-config --from-file=config-file.conf



When you run the previous command, kubectl looks for the file config-file.conf in the directory you run kubectl in. It will then store the contents of the file under the key config-file.conf in the ConfigMap (the filename is used as the map key), but you can also specify a key manually like this:

    $ kubectl create configmap my-config --from-file=customkey=config-file.conf

This command will store the file’s contents under the key customkey. As with literals, you can add multiple files by using the --from-file argument multiple times.

**Creating a ConfigMap from files in a directory**

Instead of importing each file individually, you can even import all files from a file directory:

    $ kubectl create configmap my-config --from-file=/path/to/dir



In this case, kubectl will create an individual map entry for each file in the specified directory, but only for files whose name is a valid ConfigMap key.

**Combining different options**

When creating ConfigMaps, you can use a combination of all the options mentioned here (note that these files aren’t included in the book’s code archive—you can create them yourself if you’d like to try out the command):

    $ kubectl create configmap my-config
        --from-file=foo.json
        --from-file=bar=foobar.conf
        --from-file=config-opts/
        --from-literal=some=thing

Here, you’ve created the ConfigMap from multiple sources: a whole directory, a file, another file (but stored under a custom key instead of using the filename as the key), and a literal value. Figure 7.5 shows all these sources and the resulting ConfigMap.

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/07fig05_alt.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_7-5.png" class="medium-zoom-image">

### Passing a ConfigMap entry to a container as an environment variable

How do you now get the values from this map into a pod’s container? You have three options. Let’s start with the simplest—setting an environment variable. You’ll use the valueFrom field I mentioned in section 7.3.3. The pod descriptor should look like the following listing.

```yml
apiVersion: v1
kind: Pod
metadata:
  name: fortune-env-from-configmap
spec:
  containers:
  - image: leon11sj/fortune:env
    env:
    - name: INTERVAL
      valueFrom: 
        configMapKeyRef:
          name: fortune-config
          key: sleep-interval
    name: html-generator
    volumeMounts:
    - name: html
      mountPath: /var/htdocs
  - image: nginx:alpine
    name: web-server
    volumeMounts:
    - name: html
      mountPath: /usr/share/nginx/html
      readOnly: true
    ports:
    - containerPort: 80
      protocol: TCP
  volumes:
  - name: html
    emptyDir: {}
```


    kubectl apply -f ex03-fortune-pod-env-configmap.yaml

You defined an environment variable called INTERVAL and set its value to whatever is stored in the fortune-config ConfigMap under the key sleep-interval. When the process running in the html-generator container reads the INTERVAL environment variable, it will see the value 25 (shown in figure 7.6).

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/07fig06_alt.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_7-6.png" class="medium-zoom-image">

**Referencing non-existing ConfigMaps in a pod**

You might wonder what happens if the referenced ConfigMap doesn’t exist when you create the pod. Kubernetes schedules the pod normally and tries to run its containers. The container referencing the non-existing ConfigMap will fail to start, but the other container will start normally. If you then create the missing ConfigMap, the failed container is started without requiring you to recreate the pod.

This example shows you how to decouple the configuration from the pod specification. This allows you to keep all the configuration options closely together (even for multiple pods) instead of having them splattered around the pod definition (or duplicated across multiple pod manifests).

    kubectl delete  po fortune-env-from-configmap
    kubectl delete configmap fortune-config

### Using a configMap volume to expose ConfigMap entries as files

Passing configuration options as environment variables or command-line arguments is usually used for short variable values. A ConfigMap, as you’ve seen, can also contain whole config files. When you want to expose those to the container, you can use one of the special volume types I mentioned in the previous chapter, namely a configMap volume.

A configMap volume will expose each entry of the ConfigMap as a file. The process running in the container can obtain the entry’s value by reading the contents of the file.

Although this method is mostly meant for passing large config files to the container, nothing prevents you from passing short single values this way.

**Creating the ConfigMap**

Instead of modifying your fortuneloop.sh script once again, you’ll now try a different example. You’ll use a config file to configure the Nginx web server running inside the fortune pod’s web-server container. Let’s say you want your Nginx server to compress responses it sends to the client. To enable compression, the config file for Nginx needs to look like the following listing.

    server {
      listen              80;
      server_name         www.kubia-example.com;

      gzip on;
      gzip_types text/plain application/xml;

      location / {
        root   /usr/share/nginx/html;
        index  index.html index.htm;
      }
    }

Create a new directory called `configmap-files` and store the Nginx config from the previous listing into `configmap-files/my-nginx-config.conf`. To make the ConfigMap also contain the sleep-interval entry, add a plain text file called sleep-interval to the same directory and store the number 25 in it (see figure 7.8).

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/07fig08.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_7-8.png" class="medium-zoom-image">

Now create a ConfigMap from all the files in the directory like this:

    kubectl create configmap fortune-config --from-file=configmap-files

The following listing shows what the YAML of this ConfigMap looks like.

    kubectl get configmap fortune-config -o yaml

> NOTE: The pipeline character after the colon in the first line of both entries signals that a literal multi-line value follows.

The ConfigMap contains two entries, with keys corresponding to the actual names of the files they were created from. You’ll now use the ConfigMap in both of your pod’s containers.

**Using the ConfigMap’s entries in a volume**

Creating a volume populated with the contents of a ConfigMap is as easy as creating a volume that references the ConfigMap by name and mounting the volume in a container. You already learned how to create volumes and mount them, so the only thing left to learn is how to initialize the volume with files created from a ConfigMap’s entries.

Nginx reads its config file from /etc/nginx/nginx.conf. The Nginx image already contains this file with default configuration options, which you don’t want to override, so you don’t want to replace this file as a whole. Luckily, the default config file automatically includes all .conf files in the /etc/nginx/conf.d/ subdirectory as well, so you should add your config file in there. Figure 7.9 shows what you want to achieve.

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/07fig09_alt.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_7-9.png" class="medium-zoom-image">

The pod descriptor is shown in listing 7.14 (the irrelevant parts are omitted, but you’ll find the complete file in the code archive).

```yml
apiVersion: v1
kind: Pod
metadata:
  name: fortune-configmap-volume
spec:
  containers:
  - image: leon11sj/fortune:env
    env:
    - name: INTERVAL
      valueFrom:
        configMapKeyRef:
          name: fortune-config
          key: sleep-interval
    name: html-generator
    volumeMounts:
    - name: html
      mountPath: /var/htdocs
  - image: nginx:alpine
    name: web-server
    volumeMounts:
    - name: html
      mountPath: /usr/share/nginx/html
      readOnly: true
    - name: config
      mountPath: /etc/nginx/conf.d
      readOnly: true
    ports:
      - containerPort: 80
        name: http
        protocol: TCP
  volumes:
  - name: html
    emptyDir: {}
  - name: config
    configMap:
      name: fortune-config
```

    kubectl apply -f ex04-fortune-pod-configmap-volume.yaml

This pod definition includes a volume, which references your fortune-config Config-Map. You mount the volume into the /etc/nginx/conf.d directory to make Nginx use it.

**Verifying Nginx is using the mounted config file**

The web server should now be configured to compress the responses it sends. You can verify this by enabling port-forwarding from localhost:8080 to the pod’s port 80 and checking the server’s response with curl, as shown in the following listing.

    kubectl port-forward fortune-configmap-volume 8080:80 
    curl -H "Accept-Encoding: gzip" -I localhost:8080

**Examining the mounted configMap volume’s contents**

The response shows you achieved what you wanted, but let’s look at what’s in the /etc/nginx/conf.d directory now:

    kubectl exec fortune-configmap-volume -c web-server -- ls /etc/nginx/conf.d

Both entries from the ConfigMap have been added as files to the directory. The sleep-interval entry is also included, although it has no business being there, because it’s only meant to be used by the fortuneloop container. You could create two different ConfigMaps and use one to configure the fortuneloop container and the other one to configure the web-server container. But somehow it feels wrong to use multiple ConfigMaps to configure containers of the same pod. After all, having containers in the same pod implies that the containers are closely related and should probably also be configured as a unit.

**Exposing certain ConfigMap entries in the volume**

Luckily, you can populate a configMap volume with only part of the ConfigMap’s entries—in your case, only the my-nginx-config.conf entry. This won’t affect the fortuneloop container, because you’re passing the sleep-interval entry to it through an environment variable and not through the volume.

To define which entries should be exposed as files in a configMap volume, use the volume’s items attribute as shown in the following listing.

    volumes:
      - name: config
        configMap:
          name: fortune-config
          items:
          - key: my-nginx-config.conf
            path: gzip.conf

When specifying individual entries, you need to set the filename for each individual entry, along with the entry’s key. If you run the pod from the previous listing, the /etc/nginx/conf.d directory is kept nice and clean, because it only contains the gzip.conf file and nothing else.

**Understanding that mounting a directory hides existing files in that directory**

There’s one important thing to discuss at this point. In both this and in your previous example, you mounted the volume as a directory, which means you’ve hidden any files that are stored in the /etc/nginx/conf.d directory in the container image itself.

This is generally what happens in Linux when you mount a filesystem into a nonempty directory. The directory then only contains the files from the mounted filesystem, whereas the original files in that directory are inaccessible for as long as the filesystem is mounted.

In your case, this has no terrible side effects, but imagine mounting a volume to the /etc directory, which usually contains many important files. This would most likely break the whole container, because all of the original files that should be in the /etc directory would no longer be there. If you need to add a file to a directory like /etc, you can’t use this method at all.

**Mounting individual ConfigMap entries as files without hiding other files in the directory**

Naturally, you’re now wondering how to add individual files from a ConfigMap into an existing directory without hiding existing files stored in it. An additional subPath property on the volumeMount allows you to mount either a single file or a single directory from the volume instead of mounting the whole volume. Perhaps this is easier to explain visually (see figure 7.10).

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/07fig10_alt.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_7-10.png" class="medium-zoom-image">

Say you have a configMap volume containing a myconfig.conf file, which you want to add to the /etc directory as someconfig.conf. You can use the subPath property to mount it there without affecting any other files in that directory. The relevant part of the pod definition is shown in the following listing.

    spec:
      containers:
      - image: some/image
        volumeMounts:
        - name: myvolume
          mountPath: /etc/someconfig.conf
          subPath: myconfig.conf

The subPath property can be used when mounting any kind of volume. Instead of mounting the whole volume, you can mount part of it. But this method of mounting individual files has a relatively big deficiency related to updating files. You’ll learn more about this in the following section, but first, let’s finish talking about the initial state of a configMap volume by saying a few words about file permissions.

**Setting the file permissions for files in a configMap volume**

By default, the permissions on all files in a configMap volume are set to 644 (-rw-r—r--). You can change this by setting the defaultMode property in the volume spec, as shown in the following listing.

    volumes:
      - name: config
        configMap:
          name: fortune-config
          defaultMode: "6600"

Although ConfigMaps should be used for non-sensitive configuration data, you may want to make the file readable and writable only to the user and group the file is owned by, as the example in the previous listing shows.

### Updating an app’s config without having to restart the app

We’ve said that one of the drawbacks of using environment variables or command-line arguments as a configuration source is the inability to update them while the process is running. Using a ConfigMap and exposing it through a volume brings the ability to update the configuration without having to recreate the pod or even restart the container.

When you update a ConfigMap, the files in all the volumes referencing it are updated. It’s then up to the process to detect that they’ve been changed and reload them. But Kubernetes will most likely eventually also support sending a signal to the container after updating the files.

> WARNING: Be aware that as I’m writing this, it takes a surprisingly long time for the files to be updated after you update the ConfigMap (it can take up to one whole minute).

**Editing a ConfigMap**

Let’s see how you can change a ConfigMap and have the process running in the pod reload the files exposed in the configMap volume. You’ll modify the Nginx config file from your previous example and make Nginx use the new config without restarting the pod. Try switching gzip compression off by editing the fortune-config ConfigMap with kubectl edit:

    kubectl edit configmap fortune-config

Once your editor opens, change the `gzip on` line to `gzip off`, save the file, and then close the editor. The ConfigMap is then updated, and soon afterward, the actual file in the volume is updated as well. You can confirm this by printing the contents of the file with kubectl exec:

    kubectl exec fortune-configmap-volume -c web-server -- cat /etc/nginx/conf.d/my-nginx-config.conf

If you don’t see the update yet, wait a while and try again. It takes a while for the files to get updated. Eventually, you’ll see the change in the config file, but you’ll find this has no effect on Nginx, because it doesn’t watch the files and reload them automatically.

**Signaling Nginx to reload the config**

Nginx will continue to compress its responses until you tell it to reload its config files, which you can do with the following command:

    kubectl exec fortune-configmap-volume -c web-server -- nginx -s reload

Now, if you try hitting the server again with curl, you should see the response is no longer compressed (it no longer contains the Content-Encoding: gzip header). You’ve effectively changed the app’s config without having to restart the container or recreate the pod.

    curl  -I localhost:8080

**Understanding how the files are updated atomically**

You may wonder what happens if an app can detect config file changes on its own and reloads them before Kubernetes has finished updating all the files in the configMap volume. Luckily, this can’t happen, because all the files are updated atomically, which means all updates occur at once. Kubernetes achieves this by using symbolic links.

**Understanding that files mounted into existing directories don’t get updated**

One big caveat relates to updating ConfigMap-backed volumes. If you’ve mounted a single file in the container instead of the whole volume, the file will not be updated! At least, this is true at the time of writing this chapter.

For now, if you need to add an individual file and have it updated when you update its source ConfigMap, one workaround is to mount the whole volume into a different directory and then create a symbolic link pointing to the file in question. The symlink can either be created in the container image itself, or you could create the symlink when the container starts.

**Understanding the consequences of updating a ConfigMap**

One of the most important features of containers is their immutability, which allows us to be certain that no differences exist between multiple running containers created from the same image, so is it wrong to bypass this immutability by modifying a ConfigMap used by running containers?

The main problem occurs when the app doesn’t support reloading its configuration. This results in different running instances being configured differently—those pods that are created after the ConfigMap is changed will use the new config, whereas the old pods will still use the old one. And this isn’t limited to new pods. If a pod’s container is restarted (for whatever reason), the new process will also see the new config. Therefore, if the app doesn’t reload its config automatically, modifying an existing ConfigMap (while pods are using it) may not be a good idea.

If the app does support reloading, modifying the ConfigMap usually isn’t such a big deal, but you do need to be aware that because files in the ConfigMap volumes aren’t updated synchronously across all running instances, the files in individual pods may be out of sync for up to a whole minute.

    kubectl delete pod fortune-configmap-volume
    kubectl delete configmaps fortune-config



## Using Secrets to pass sensitive data to containers

All the information you’ve passed to your containers so far is regular, non-sensitive configuration data that doesn’t need to be kept secure. But as we mentioned at the start of the chapter, the config usually also includes sensitive information, such as credentials and private encryption keys, which need to be kept secure.

### Introducing Secrets

To store and distribute such information, Kubernetes provides a separate object called a Secret. Secrets are much like ConfigMaps—they’re also maps that hold key-value pairs. They can be used the same way as a ConfigMap. You can
- Pass Secret entries to the container as environment variables
- Expose Secret entries as files in a volume

Kubernetes helps keep your Secrets safe by making sure **each Secret is only distributed to the nodes that run the pods that need access to the Secret**. Also, on the nodes themselves, **Secrets are always stored in memory** and never written to physical storage, which would require wiping the disks after deleting the Secrets from them.

On the master node itself (more specifically in etcd), Secrets used to be stored in unencrypted form, which meant the master node needs to be secured to keep the sensitive data stored in Secrets secure. This didn’t only include keeping the etcd storage secure, but also preventing unauthorized users from using the API server, because anyone who can create pods can mount the Secret into the pod and gain access to the sensitive data through it. From Kubernetes version 1.7, **etcd stores Secrets in encrypted form, making the system much more secure**. Because of this, it’s imperative you properly choose when to use a Secret or a ConfigMap. Choosing between them is simple:

- Use a ConfigMap to store non-sensitive, plain configuration data.
- Use a Secret to store any data that is sensitive in nature and needs to be kept under key. If a config file includes both sensitive and not-sensitive data, you should store the file in a Secret.

    kubectl get secrets

### Creating a Secret

Now, you’ll create your own little Secret. You’ll improve your fortune-serving Nginx container by configuring it to also serve HTTPS traffic. For this, you need to create a certificate and a private key. The private key needs to be kept secure, so you’ll put it and the certificate into a Secret.

First, generate the certificate and private key files (do this on your local machine). You can also use the files in the book’s code archive (the cert and key files are in the `fortune-https` directory):

    openssl genrsa -out https.key 2048
    openssl req -new -x509 -key https.key -out https.cert -days 3650 -subj /CN=www.kubia-example.com

Now, to help better demonstrate a few things about Secrets, create an additional dummy file called foo and make it contain the string bar. You’ll understand why you need to do this in a moment or two:


    echo bar > foo

Now you can use kubectl create secret to create a Secret from the three files:

    kubectl create secret generic fortune-https --from-file=https.key --from-file=https.cert --from-file=foo




This isn’t very different from creating ConfigMaps. In this case, you’re creating a generic Secret called fortune-https and including two entries in it (https.key with the contents of the https.key file and likewise for the https.cert key/file). As you learned earlier, you could also include the whole directory with --from-file=fortune-https instead of specifying each file individually.

> You’re creating a generic Secret, but you could also have created a tls Secret with the kubectl create secret tls command, as you did in chapter 5. This would create the Secret with different entry names, though.

### Comparing ConfigMaps and Secrets

Secrets and ConfigMaps have a pretty big difference. This is what drove Kubernetes developers to create ConfigMaps after Kubernetes had already supported Secrets for a while. The following listing shows the YAML of the Secret you created.

    kubectl get secret fortune-https -o yaml

Notice the difference? The contents of a Secret’s entries are shown as Base64-encoded strings, whereas those of a ConfigMap are shown in clear text. This initially made working with Secrets in YAML and JSON manifests a bit more painful, because you had to encode and decode them when setting and reading their entries.

**Using Secrets for binary data**

The reason for using Base64 encoding is simple. A Secret’s entries can contain binary values, not only plain-text. Base64 encoding allows you to include the binary data in YAML or JSON, which are both plain-text formats.

> You can use Secrets even for non-sensitive binary data, but be aware that the maximum size of a Secret is limited to 1MB.

**Reading a Secret’s entry in a pod**

When you expose the Secret to a container through a secret volume, the value of the Secret entry is decoded and written to the file in its actual form (regardless if it’s plain text or binary). The same is also true when exposing the Secret entry through an environment variable. In both cases, the app doesn’t need to decode it, but can read the file’s contents or look up the environment variable value and use it directly.

### Using the Secret in a pod

With your fortune-https Secret containing both the cert and key files, all you need to do now is configure Nginx to use them.

After the text editor opens, modify the part that defines the contents of the my-nginx-config.conf entry so it looks like the following listing.

    server {
        listen              80;
        listen              443 ssl;
        server_name         www.kubia-example.com;
        ssl_certificate     certs/https.cert;
        ssl_certificate_key certs/https.key;
        ssl_protocols       TLSv1 TLSv1.1 TLSv1.2;
        ssl_ciphers         HIGH:!aNULL:!MD5;

        location / {
            root   /usr/share/nginx/html;
            index  index.html index.htm;
        }

    }

This configures the server to read the certificate and key files from /etc/nginx/certs, so you’ll need to mount the secret volume there.

    kubectl create configmap fortune-config --from-file=configmap-files-https
    

**Mounting the fortune-https Secret in a pod**

Next, you’ll create a new fortune-https pod and mount the secret volume holding the certificate and key into the proper location in the web-server container, as shown in the following listing.

```yml
apiVersion: v1
kind: Pod
metadata:
  name: fortune-https
spec:
  containers:
  - image: leon11sj/fortune:env
    name: html-generator
    env:
    - name: INTERVAL
      valueFrom:
        configMapKeyRef:
          name: fortune-config
          key: sleep-interval
    volumeMounts:
    - name: html
      mountPath: /var/htdocs
  - image: nginx:alpine
    name: web-server
    volumeMounts:
    - name: html
      mountPath: /usr/share/nginx/html
      readOnly: true
    - name: config
      mountPath: /etc/nginx/conf.d
      readOnly: true
    - name: certs
      mountPath: /etc/nginx/certs/
      readOnly: true
    ports:
    - containerPort: 80
    - containerPort: 443
  volumes:
  - name: html
    emptyDir: {}
  - name: config
    configMap:
      name: fortune-config
      items:
      - key: ssl.conf
        path: https.conf
  - name: certs
    secret:
      secretName: fortune-https
```

Much is going on in this pod descriptor, so let me help you visualize it. Figure 7.12 shows the components defined in the YAML. The default-token Secret, volume, and volume mount, which aren’t part of the YAML, but are added to your pod automatically, aren’t shown in the figure.

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/07fig12_alt.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_7-12.png" class="medium-zoom-image">

> NOTE: Like configMap volumes, secret volumes also support specifying file permissions for the files exposed in the volume through the defaultMode property.

    kubectl apply -f ex05-fortune-pod-https.yaml

**Testing whether Nginx is using the cert and key from the Secret**

Once the pod is running, you can see if it’s serving HTTPS traffic by opening a port-forward tunnel to the pod’s port 443 and using it to send a request to the server with curl:

    kubectl port-forward fortune-https 8443:443 

    curl https://localhost:8443 -k

If you configured the server properly, you should get a response. You can check the server’s certificate to see if it matches the one you generated earlier. This can also be done with curl by turning on verbose logging using the -v option, as shown in the following listing.

    curl https://localhost:8443 -k -v

**Understanding secret volumes are stored in memory**

You successfully delivered your certificate and private key to your container by mounting a secret volume in its directory tree at /etc/nginx/certs. The secret volume uses an in-memory filesystem (tmpfs) for the Secret files. 

    kubectl exec fortune-https -c web-server -- mount | grep certs

Because tmpfs is used, the sensitive data stored in the Secret is never written to disk, where it could be compromised.



**Exposing a Secret’s entries through environment variables**

Even though Kubernetes enables you to expose Secrets through environment variables, it may not be the best idea to use this feature. Applications usually dump environment variables in error reports or even write them to the application log at startup, which may unintentionally expose them. Additionally, child processes inherit all the environment variables of the parent process, so if your app runs a third-party binary, you have no way of knowing what happens with your secret data.

> TIP: Think twice before using environment variables to pass your Secrets to your container, because they may get exposed inadvertently. To be safe, always use secret volumes for exposing Secrets.